<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/ElevatorSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simplified simulation of an elevator transporting people between floors in a building

**To Do:**
- multiple elevators
- add capacity (multiple people per elevator)
- add routing optimization (pickup in between)
- More realistic demand
  - Rush hours
  - More frequency for 1st floor

In [18]:
!pip install simpy --quiet
import simpy

In [19]:
# elevator.py

from collections import deque

class Elevator:
    def __init__(self, env: simpy.Environment, floors: tuple[int], speed_floors_per_sec: float):
        """
        Elevator agent.

        Args:
            env: SimPy environment
            floors: Valid floor numbers (e.g., (1, 2, 3, ..., 10))
            speed_floors_per_sec: Constant speed of elevator in floors per second
        """
        self.env = env
        self.floors = floors
        self.speed = speed_floors_per_sec

        self.current_floor = floors[0]
        self.task_queue = deque()
        self.moving = False

        # Start the elevator process
        self.process = env.process(self.run())

    def add_task(self, target_floor: int):
        """
        Enqueue a request to move to a specific floor.
        """
        if target_floor not in self.floors:
            raise ValueError(f"Invalid floor: {target_floor}")
        self.task_queue.append(target_floor)

    def hold(self, duration: float):
        """
        Elevator remains idle at current floor for a set duration.
        """
        yield self.env.timeout(duration)

    def move_to(self, target_floor: int):
        """
        Simulates elevator travel from current floor to target_floor.
        Uses constant speed to compute duration.
        """
        # Calculate travel time
        floor_diff = abs(self.current_floor - target_floor)
        if floor_diff == 0:
            return  # Already at floor
        travel_time = floor_diff / self.speed

        # Move event
        self.moving = True
        print(f"[{self.env.now:.1f}] Elevator starting move from {self.current_floor} to {target_floor}")
        yield self.env.timeout(travel_time)

        self.current_floor = target_floor
        self.moving = False
        print(f"[{self.env.now:.1f}] Elevator arrived at floor {self.current_floor}")

    def run(self):
        """
        Elevator main loop: process queued tasks in FIFO order.
        """
        while True:
            if self.task_queue:

              # Get next task
              next_floor = self.task_queue.popleft()
              print(f"[{self.env.now:.1f}] Elevator processing request to floor {next_floor}")

              # Move if necesary
              if next_floor == self.current_floor and not self.moving:
                print(f"[{self.env.now:.1f}] Elevator is already at floor {next_floor}")
              else:
                yield self.env.process(self.move_to(next_floor))
                yield self.env.process(self.hold(1.0)) # hold briefly after arrival
            else:
                # Idle when no tasks
                yield self.env.timeout(0.5)


In [20]:
# demandGenerator.py

import simpy
import random

class DemandGenerator:
    def __init__(self, env: simpy.Environment, floors: tuple[int], elevator, lambda_: float):
        """
        Generates elevator demand at random intervals.

        Args:
            env: SimPy environment
            floors: Valid floor numbers
            elevator: Reference to the Elevator instance
            lambda_: Mean arrival interval (Exponential distribution)
        """
        self.env = env
        self.floors = floors
        self.elevator = elevator
        self.lambda_ = lambda_

        # Start the generator process
        self.process = env.process(self.run())

    def generate_interarrival_time(self) -> float:
        """
        Samples the next interarrival time from an exponential distribution.
        """
        return random.expovariate(1 / self.lambda_)

    def generate_origin_destination(self) -> tuple[int, int]:
        """
        Randomly selects origin and destination floors (must differ).
        """
        origin = random.choice(self.floors)
        destination = random.choice([f for f in self.floors if f != origin])
        return origin, destination

    def run(self):
        """
        Main loop: generates demand at stochastic intervals and sends tasks to the elevator.
        """
        while True:
            # Wait until next demand
            interarrival_time = self.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)

            # Generate a random request
            origin, destination = self.generate_origin_destination()

            # In this simplified version, elevator just gets a task to go to origin then to destination
            print(f"[{self.env.now:.1f}] Request: from {origin} to {destination}")
            self.elevator.add_task(origin)
            self.elevator.add_task(destination)


In [21]:
# simulation.py

#from elevator import Elevator
#from demand_generator import DemandGenerator

class Simulation:
    def __init__(
        self,
        sim_time: float,
        floors: tuple[int],
        speed_floors_per_sec: float,
        lambda_: float
    ):
        """
        Main simulation controller.

        Args:
            sim_time: Total duration of the simulation (in seconds)
            floors: Valid floor numbers (e.g., (1, 2, ..., 10))
            speed_floors_per_sec: Elevator travel speed
            lambda_: Average time between user requests (Poisson process)
        """
        self.sim_time = sim_time
        self.env = simpy.Environment()

        # Initialize elevator and demand generator
        self.elevator = Elevator(
            env=self.env,
            floors=floors,
            speed_floors_per_sec=speed_floors_per_sec
        )

        self.demand_generator = DemandGenerator(
            env=self.env,
            floors=floors,
            elevator=self.elevator,
            lambda_=lambda_
        )

    def run(self):
        """
        Runs the simulation.
        """
        self.env.run(until=self.sim_time)


In [22]:
# run

if __name__ == "__main__":
    sim = Simulation(
        sim_time=100,                  # Run for 100 simulated seconds
        floors=tuple(range(1, 6)),     # Floors 1 to 5
        speed_floors_per_sec=1.0,      # 1 floor per second
        lambda_=10                     # Average of 1 request every 10 seconds
    )
    sim.run()


[7.3] Request: from 3 to 5
[7.5] Elevator processing request to floor 3
[7.5] Elevator starting move from 1 to 3
[9.5] Elevator arrived at floor 3
[10.5] Elevator processing request to floor 5
[10.5] Elevator starting move from 3 to 5
[10.5] Request: from 2 to 5
[11.3] Request: from 5 to 4
[12.1] Request: from 3 to 4
[12.5] Elevator arrived at floor 5
[13.5] Elevator processing request to floor 2
[13.5] Elevator starting move from 5 to 2
[16.2] Request: from 2 to 3
[16.5] Elevator arrived at floor 2
[17.5] Elevator processing request to floor 5
[17.5] Elevator starting move from 2 to 5
[20.5] Elevator arrived at floor 5
[21.5] Elevator processing request to floor 5
[21.5] Elevator is already at floor 5
[21.5] Elevator processing request to floor 4
[21.5] Elevator starting move from 5 to 4
[22.5] Elevator arrived at floor 4
[23.5] Elevator processing request to floor 3
[23.5] Elevator starting move from 4 to 3
[24.5] Elevator arrived at floor 3
[25.5] Elevator processing request to floo